In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from transformers import ViTForImageClassification
from PIL import Image
import random
import numpy as np

In [2]:
# Define Random Cut Augmentation
class RandomCutTransform:
    def __init__(self, cut_size):
        self.cut_size = cut_size

    def __call__(self, img):
        w, h = img.size
        x, y = random.randint(0, w - self.cut_size), random.randint(0, h - self.cut_size)
        img_np = np.array(img)
        img_np[y:y+self.cut_size, x:x+self.cut_size, :] = img_np.mean(axis=(0, 1))
        return Image.fromarray(img_np)

# Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #RandomCutTransform(cut_size=20),  # Example cut size
    transforms.ToTensor(),
])

In [14]:
# Load the entire dataset
dataset_folder = "dataset_ebs_test_cropped_aug"
full_dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)

# Calculate split sizes for training and testing
total_size = len(full_dataset)
train_size = int(0.7 * total_size)  # 70% of the dataset
test_size = total_size - train_size  # Remaining 30% of the dataset

In [4]:
# Randomly split the dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize Vision Transformer Model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

# Define Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Number of Epochs
num_epochs = 10  # Adjust as needed


In [5]:
# Training Loop
for epoch in range(num_epochs):
    model.train()
    
    # Variables to track training performance
    train_loss = 0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)
        train_loss += loss.item()
        _, predicted = torch.max(outputs.logits.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_accuracy = 100 * correct_train / total_train

    # Evaluate on the test set
    model.eval()
    test_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs.logits, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.logits.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = 100 * correct_test / total_test

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/total_train:.4f}, Train Acc: {train_accuracy:.2f}%, Test Loss: {test_loss/total_test:.4f}, Test Acc: {test_accuracy:.2f}%')


Epoch 1/10, Train Loss: 0.0470, Train Acc: 50.31%, Test Loss: 0.0233, Test Acc: 69.19%
Epoch 2/10, Train Loss: 0.0218, Train Acc: 61.51%, Test Loss: 0.0224, Test Acc: 69.19%
Epoch 3/10, Train Loss: 0.0229, Train Acc: 57.03%, Test Loss: 0.0238, Test Acc: 69.19%
Epoch 4/10, Train Loss: 0.0242, Train Acc: 58.66%, Test Loss: 0.0205, Test Acc: 69.19%
Epoch 5/10, Train Loss: 0.0225, Train Acc: 57.43%, Test Loss: 0.0232, Test Acc: 30.81%
Epoch 6/10, Train Loss: 0.0228, Train Acc: 55.40%, Test Loss: 0.0205, Test Acc: 69.19%
Epoch 7/10, Train Loss: 0.0229, Train Acc: 57.43%, Test Loss: 0.0213, Test Acc: 69.19%
Epoch 8/10, Train Loss: 0.0215, Train Acc: 65.58%, Test Loss: 0.0206, Test Acc: 69.19%
Epoch 9/10, Train Loss: 0.0215, Train Acc: 65.58%, Test Loss: 0.0208, Test Acc: 69.19%
Epoch 10/10, Train Loss: 0.0213, Train Acc: 65.58%, Test Loss: 0.0208, Test Acc: 69.19%


In [6]:

# ... Add code for further evaluation, saving the model, etc. ...


In [7]:
class_names = full_dataset.classes
num_classes = len(class_names)




In [16]:
print("Training Setup Summary:")
print(f"Dataset Folder: {dataset_folder}")
print(f"Training-Testing Ratio: {train_size}:{test_size}")
print(f"Class Names: {class_names}")
print(f"Number of Epochs: {num_epochs}")


Training Setup Summary:
Dataset Folder: dataset_ebs_test_cropped_aug
Training-Testing Ratio: 491:211
Class Names: ['goods', 'rejects']
Number of Epochs: 10
